In [ ]:
# https://www.eia.gov/opendata/bulk/manifest.txt
# https://www.eia.gov/developer/

import requests
import json

url = 'https://www.eia.gov/opendata/bulk/manifest.txt'
response = requests.get(url)
data = json.loads(response.text)

# Pretty print the JSON data
print(json.dumps(data, indent=4))

In [ ]:
import pandas as pd

# Load the JSON data into a Pandas DataFrame
df = pd.DataFrame.from_dict(data['dataset'], orient='index')

# Focus on international data
df = df[df['name'].str.contains('international', case=False)]
# Convert the 'date' column to datetime format
df['last_updated'] = pd.to_datetime(df['last_updated']).dt.strftime('%Y-%m-%d')


for _, row in df.iterrows():
    print(list(row))

df

In [ ]:
# https://api.eia.gov/v2/international/data/?frequency=annual&data[0]=value&facets[activityId][]=34&facets[productId][]=4701&facets[countryRegionId][]=WORL&facets[unit][]=BDOLPPP&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000


import requests

# Set up the API endpoint URL
url = "https://api.eia.gov/v2/international/data/"

# Set up the request parameters
params = {
    "api_key": "TYUMyndQZkshGBZTAM0tUfslaM1pvIctp1bcK7iV",
    "offset": "0",
    "frequency": "annual",
    "data[0]": "value"
}

df_result = pd.DataFrame()

for i in range(0, 200, 10):
    params['start'] = str(1900+i+1)
    params['end'] = str(1900+i+10)
    # Send the GET request and print the response url
    response = requests.get(url, params=params)
    print(response.url)
    json_response = json.loads(response.text)['response']
    # Pretty print the JSON data
    #print(json.dumps(json_data, indent=4))
    
    if 'warning' in json_response:
        print('EXCEEDED 5000 LIMIT')
        break

    df = pd.json_normalize(json_response['data'])
    if (not df.empty):
        df_result = pd.concat([df_result, df], ignore_index=True)      

# TODO : CLEAN DATA
df_result.to_csv('../../processed/eia_api_intl.csv', index=False)

df_result.head()